<a href="https://colab.research.google.com/github/vikasmech/TMDSI/blob/main/Data_prep_EDA_TMDSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls

drive/  sample_data/


In [ ]:
cd drive/

/content/drive


In [ ]:
ls

MyDrive/


In [ ]:
cd MyDrive/

/content/drive/MyDrive


In [ ]:
cd DSI/

/content/drive/MyDrive/DSI


In [ ]:
cd DATA/

/content/drive/MyDrive/DSI/DATA


In [ ]:
pwd

'/content/drive/MyDrive/DSI/DATA'

In [ ]:
DATA_DIR = "/content/drive/MyDrive/DSI/DATA"

In [ ]:
ls

nq-counts.json  nq-train.tsv  nq-validation.tsv


In [ ]:
#%%bash
#gzip -d v1.0-simplified_nq-dev-all.jsonl.gz

In [ ]:
ls -rltah

total 7.1G
-rw------- 1 root root 6.9G Mar 28 13:50 nq-train.tsv
-rw------- 1 root root 164M Mar 28 13:58 nq-validation.tsv
-rw------- 1 root root   36 Mar 28 14:01 nq-counts.json


In [ ]:
%tensorflow_version 2.x
import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

In [ ]:
!git clone https://github.com/google-research-datasets/natural-questions.git

Cloning into 'natural-questions'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 92 (delta 4), reused 8 (delta 3), pack-reused 79
Unpacking objects: 100% (92/92), done.


In [ ]:
!ls

natural-questions  nq-counts.json  nq-train.tsv  nq-validation.tsv


In [ ]:
cd natural-questions/

/content/drive/MyDrive/DSI/DATA/natural-questions


In [ ]:
ls -rltah

total 110K
-rw------- 1 root root 9.8K Mar 28 19:40 README.md
-rw------- 1 root root 7.0K Mar 28 19:40 nq_eval_test.py
-rw------- 1 root root  16K Mar 28 19:40 nq_eval.py
-rw------- 1 root root  12K Mar 28 19:40 nq_browser.py
-rw------- 1 root root 4.2K Mar 28 19:40 make_test_data.py
-rw------- 1 root root  12K Mar 28 19:40 LICENSE
-rw------- 1 root root  616 Mar 28 19:40 __init__.py
-rw------- 1 root root 2.3K Mar 28 19:40 eval_utils_test.py
-rw------- 1 root root  11K Mar 28 19:40 eval_utils.py
-rw------- 1 root root 1.1K Mar 28 19:40 CONTRIBUTING.md
-rw------- 1 root root 3.8K Mar 28 19:40 competition.md
-rw------- 1 root root 2.8K Mar 28 19:40 toy_example.md
-rw------- 1 root root 5.1K Mar 28 19:40 text_utils.py
-rw------- 1 root root 1.2K Mar 28 19:40 test_docker.sh
drwx------ 2 root root 4.0K Mar 28 19:40 templates/
drwx------ 2 root root 4.0K Mar 28 19:40 static/
-rw------- 1 root root 2.6K Mar 28 19:40 simplify_nq_data.py
drwx------ 2 root root 4.0K Mar 28 19:40 nq_open/
drwx--

In [ ]:
import gzip
import json

# Public directory of Natural Questions data on GCS.
NQ_JSONL_DIR = "gs://natural_questions/v1.0-simplified/"
NQ_SPLIT_FNAMES = {
    "train": "simplified-nq-train.jsonl.gz",
    "validation": "nq-dev-all.jsonl.gz"
}
nq_counts_path = os.path.join(DATA_DIR, "nq-counts.json")
nq_tsv_path = {
    "train": os.path.join(DATA_DIR, "nq-train.tsv"),
    "validation": os.path.join(DATA_DIR, "nq-validation.tsv")
}

def nq_jsonl_to_tsv(in_fname, out_fname):

  def extract_answer(tokens, span):
    """Reconstruct answer from token span and remove extra spaces."""
    start, end = span["start_token"], span["end_token"]
    ans = " ".join(tokens[start:end])
    # Remove incorrect spacing around punctuation.
    ans = ans.replace(" ,", ",").replace(" .", ".").replace(" %", "%")
    ans = ans.replace(" - ", "-").replace(" : ", ":").replace(" / ", "/")
    ans = ans.replace("( ", "(").replace(" )", ")")
    ans = ans.replace("`` ", "\"").replace(" ''", "\"")
    ans = ans.replace(" 's", "'s").replace("s ' ", "s' ")
    return ans
  def clean_document(tokens):
    """clean document from token"""
    
    doc = " ".join(tokens)
    # Remove incorrect spacing around punctuation.
    doc = doc.replace(" ,", ",").replace(" .", ".").replace(" %", "%")
    doc = doc.replace(" - ", "-").replace(" : ", ":").replace(" / ", "/")
    doc = doc.replace("( ", "(").replace(" )", ")")
    doc = doc.replace("`` ", "\"").replace(" ''", "\"")
    doc = doc.replace(" 's", "'s").replace("s ' ", "s' ")
    return doc

  count = 0
  with tf.io.gfile.GFile(in_fname, "rb") as infile,\
       tf.io.gfile.GFile(out_fname, "w") as outfile:
    for line in gzip.open(infile):
      ex = json.loads(line)
      # Remove any examples with more than one answer.
      if len(ex['annotations'][0]['short_answers']) != 1:
        continue
      # Questions in NQ do not include a question mark.
      question = ex["question_text"] + "?"
      answer_span = ex['annotations'][0]['short_answers'][0]
      # Handle the two document formats in NQ (tokens or text).
      if "document_tokens" in ex:
        tokens = [t["token"] for t in ex["document_tokens"]]
      elif "document_text" in ex:
        tokens = ex["document_text"].split(" ")
      answer = extract_answer(tokens, answer_span)
      tokens = clean_document(tokens)
      # Write this line as <question>\t<answer>
      outfile.write("%s\t%s\t%s\n" % (question,tokens,answer))
      count += 1
      tf.logging.log_every_n(
          tf.logging.INFO,
          "Wrote %d examples to %s." % (count, out_fname),
          1000)
    return count

if tf.io.gfile.exists(nq_counts_path):
  # Used cached data and counts.
  tf.logging.info("Loading NQ from cache.")
  num_nq_examples = json.load(tf.io.gfile.GFile(nq_counts_path))
else:
  # Create TSVs and get counts.
  tf.logging.info("Generating NQ TSVs.")
  num_nq_examples = {}
  for split, fname in NQ_SPLIT_FNAMES.items():
    num_nq_examples[split] = nq_jsonl_to_tsv(
        os.path.join(NQ_JSONL_DIR, fname), nq_tsv_path[split])
  json.dump(num_nq_examples, tf.io.gfile.GFile(nq_counts_path, "w"))

INFO:tensorflow:Generating NQ TSVs.
INFO:tensorflow:Wrote 1 examples to /content/drive/MyDrive/DSI/DATA/nq-train.tsv.
INFO:tensorflow:Wrote 1001 examples to /content/drive/MyDrive/DSI/DATA/nq-train.tsv.
INFO:tensorflow:Wrote 2001 examples to /content/drive/MyDrive/DSI/DATA/nq-train.tsv.
INFO:tensorflow:Wrote 3001 examples to /content/drive/MyDrive/DSI/DATA/nq-train.tsv.
INFO:tensorflow:Wrote 4001 examples to /content/drive/MyDrive/DSI/DATA/nq-train.tsv.
INFO:tensorflow:Wrote 5001 examples to /content/drive/MyDrive/DSI/DATA/nq-train.tsv.
INFO:tensorflow:Wrote 6001 examples to /content/drive/MyDrive/DSI/DATA/nq-train.tsv.
INFO:tensorflow:Wrote 7001 examples to /content/drive/MyDrive/DSI/DATA/nq-train.tsv.
INFO:tensorflow:Wrote 8001 examples to /content/drive/MyDrive/DSI/DATA/nq-train.tsv.
INFO:tensorflow:Wrote 9001 examples to /content/drive/MyDrive/DSI/DATA/nq-train.tsv.
INFO:tensorflow:Wrote 10001 examples to /content/drive/MyDrive/DSI/DATA/nq-train.tsv.
INFO:tensorflow:Wrote 11001 exa

In [ ]:
cd ..

/content/drive/MyDrive/DSI/DATA


In [ ]:
!head nq-train.tsv

which is the most common use of opt-in e-mail marketing?	Email marketing-Wikipedia <H1> Email marketing </H1> Jump to:navigation, search <Table> <Tr> <Td> </Td> <Td> (hide) This article has multiple issues. Please help improve it or discuss these issues on the talk page. (Learn how and when to remove these template messages) <Table> <Tr> <Td> </Td> <Td> This article needs additional citations for verification. Please help improve this article by adding citations to reliable sources. Unsourced material may be challenged and removed. (September 2014) (Learn how and when to remove this template message) </Td> </Tr> </Table> <Table> <Tr> <Td> </Td> <Td> This article possibly contains original research. Please improve it by verifying the claims made and adding inline citations. Statements consisting only of original research should be removed. (January 2015) (Learn how and when to remove this template message) </Td> </Tr> </Table> (Learn how and when to remove this template message) </Td> <

In [ ]:
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
r"""Utilities to simplify the canonical NQ data.

The canonical NQ data contains the HTML of each Wikipedia page along with a
sequence of tokens on that page, each of which is indexed into the HTML.

Many users will not want to use the HTML at all, and this file provides
utilities to extract only the text into a new record of the form:

  {
    "example_id": 3902,
    "document_url": "http://wikipedia.org/en/strings"
    "question_text": "what is a string",
    "document_text": "<P> A string is a list of characters in order . </P>",
    "annotations": [{
      "long_answer": { "start_token": 0, "end_token": 12 },
      "short_answers": [{ "start_token": 5, "end_token": 8 }],
      "yes_no_answer": "NONE",
    }],
    "long_answer_candidates": [
      {"start_token": 0, "end_token": 12, "top_level": True}
    ]
  }

which leads to a much smaller training set (4.4Gb instead of 41Gb).

In this representation, the [start, end) indices are into the blank separated
sequence of tokens. So, answer spans can be extracted using the following
snippet:

  " ".join(example["document_text"].split(" ")[`start_token`:`end_token`]).

WARNING: Use `split(" ")` instead of `split()` to avoid complications from
  characters such as `\u180e` which may or may not be recognized as a whitespace
  character depending on your python version.

To avoid complications at test time, we do not provide a simplified version
of the development data, and there is no simplified version of the hidden test
set. If you rely on the simplified data, then you must call the
`simplify_nq_example` function below on every example that is passed in at test
time.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import re


def get_nq_tokens(simplified_nq_example):
  """Returns list of blank separated tokens."""

  if "document_text" not in simplified_nq_example:
    raise ValueError("`get_nq_tokens` should be called on a simplified NQ"
                     "example that contains the `document_text` field.")

  return simplified_nq_example["document_text"].split(" ")


def simplify_nq_example(nq_example):
  r"""Returns dictionary with blank separated tokens in `document_text` field.

  Removes byte offsets from annotations, and removes `document_html` and
  `document_tokens` fields. All annotations in the ouput are represented as
  [start_token, end_token) offsets into the blank separated tokens in the
  `document_text` field.

  WARNING: Tokens are separated by a single blank character. Do not split on
    arbitrary whitespace since different implementations have different
    treatments of some unicode characters such as \u180e.

  Args:
    nq_example: Dictionary containing original NQ example fields.

  Returns:
    Dictionary containing `document_text` field, not containing
    `document_tokens` or `document_html`, and with all annotations represented
    as [`start_token`, `end_token`) offsets into the space separated sequence.
  """

  def _clean_token(token):
    """Returns token in which blanks are replaced with underscores.

    HTML table cell openers may contain blanks if they span multiple columns.
    There are also a very few unicode characters that are prepended with blanks.

    Args:
      token: Dictionary representation of token in original NQ format.

    Returns:
      String token.
    """
    return re.sub(u" ", "_", token["token"])

  text = " ".join([_clean_token(t) for t in nq_example["document_tokens"]])

  def _remove_html_byte_offsets(span):
    if "start_byte" in span:
      del span["start_byte"]

    if "end_byte" in span:
      del span["end_byte"]

    return span

  def _clean_annotation(annotation):
    annotation["long_answer"] = _remove_html_byte_offsets(
        annotation["long_answer"])
    annotation["short_answers"] = [
        _remove_html_byte_offsets(sa) for sa in annotation["short_answers"]
    ]
    return annotation

  simplified_nq_example = {
      "question_text": nq_example["question_text"],
      "example_id": nq_example["example_id"],
      "document_url": nq_example["document_url"],
      "document_text": text,
      "long_answer_candidates": [
          _remove_html_byte_offsets(c)
          for c in nq_example["long_answer_candidates"]
      ],
      "annotations": [_clean_annotation(a) for a in nq_example["annotations"]]
  }

  if len(get_nq_tokens(simplified_nq_example)) != len(
      nq_example["document_tokens"]):
    raise ValueError("Incorrect number of tokens.")

  return simplified_nq_example